# Generating Representative Sentences from Web Crawling Data

In this notebook, we try to implement methods that generating sentences from source-articles and term-frequency data.

## DATA API
- PTT
    + Deployeed on: http://52.26.156.12:8000/
    + Two functions:
        * get_json: return json as a string
            - http://52.26.156.12:8000/get_json/?item_type=termscores&item_date=2021-11-09
            - http://52.26.156.12:8000/get_json/?item_type=posts&item_date=2021-11-09
        * get_file: return file object (.csv)

In [9]:
import requests
import json

posts = requests.get('http://52.26.156.12:4001/get_json/?item_type=posts')
tfreq = requests.get('http://52.26.156.12:4001/get_json/?item_type=termscores')

print(posts)

<Response [200]>


In [12]:
import pandas as pd

posts = pd.DataFrame(eval(posts.text))
print(posts.head())

                   id                          title    author  \
0  M.1638329725.A.75F       [新聞] 多圖／高嘉瑜遭男友林秉樞暴打「驚悚傷勢     duise   
1  M.1638365035.A.B3D        [新聞] 獨／男友視角照帶風向「沒品維尼」真實    qishuv   
2  M.1638357037.A.FD5  Re: [新聞] 新女友？高嘉瑜男友飯店被捕 1長髮女陪伴   zoe5566   
3  M.1638314104.A.853        [新聞] 打到高嘉瑜全身傷　「男友老家整晚亮燈  hsnugear   
4  M.1638327947.A.546      [新聞] 新女友？高嘉瑜男友飯店被捕 1長髮女陪伴    murray   

                                                link  \
0  https://www.ptt.cc/bbs/Gossiping/M.1638329725....   
1  https://www.ptt.cc/bbs/Gossiping/M.1638365035....   
2  https://www.ptt.cc/bbs/Gossiping/M.1638357037....   
3  https://www.ptt.cc/bbs/Gossiping/M.1638314104....   
4  https://www.ptt.cc/bbs/Gossiping/M.1638327947....   

                                             content  n_comments  n_push  \
0  \n1.媒體來源:\n三立\n\n2.記者署名:\n許皓婷\n\n3.完整新聞標題:\n多圖...        1290     801   
1  \n獨／男友視角照帶風向「沒品維尼」真實身分是他！暗黑操作驚曝光\n\n2021年12月01...        1214     745   
2  \n大家好，我大麻莉絲啦\n關於這個林秉樞我剛好也是略懂略懂\n大概是

In [35]:
#tfreq = pd.DataFrame(eval(tfreq.text))
print(tfreq.head(10))

  term      score
0  高嘉瑜  66.650234
1   林秉  36.324876
2   新北  24.005165
3   施暴  23.549037
4   林男  16.387003
5  蔡詩萍  14.176818
6  民進黨  13.461002
7   台北  13.088543
8  陳其邁  12.549524
9   老公  12.231784


In [14]:
print(posts['content'].iloc[0])


1.媒體來源:
三立

2.記者署名:
許皓婷

3.完整新聞標題:
多圖／高嘉瑜遭男友林秉樞暴打　「驚悚傷勢照」曝光

4.完整新聞內文:
民進黨立委高嘉瑜驚傳遭男友林秉樞虐打，導致她右臉、頭部、四肢多處挫傷，引發社會
關注，她昨日向男方提告傷害罪後，今（1日）在律師李永裕陪同下，在立法院議場廣場
召開記者會，對外說明家暴始末。

高嘉瑜遭到男友林秉樞暴打，她在遭毆後3天前往醫院驗傷，只見她除了右眼嚴重瘀青、
嘴唇、嘴角都有傷口，右臉腫了半邊，脖子還有出現勒痕，雙腳和雙手關節也有傷勢，她
的兩腳膝蓋更破皮、流血。




In [52]:
def article_to_sentences(article, min_length=5):
    ''' Parse an article into a list sentences. '''
    # Define sentence-break symbols
    bs = ['\n','，','。','；','！','？','「','」','.',':','（','）','／','　','~','：']
    # Loop through the article character-by-character
    sentences = []
    tmp = []
    for char in article:
        if not char in bs:
            tmp.append(char)
        else:
            if len(tmp)>=min_length:
                sentences.append(''.join(tmp).strip())
            tmp = []
    return(sentences)

sentences = article_to_sentences(posts['content'].iloc[0])
print(sentences)

['完整新聞標題', '高嘉瑜遭男友林秉樞暴打', '驚悚傷勢照', '完整新聞內文', '民進黨立委高嘉瑜驚傳遭男友林秉樞虐打', '導致她右臉、頭部、四肢多處挫傷', '她昨日向男方提告傷害罪後', '在律師李永裕陪同下', '在立法院議場廣場', '召開記者會', '對外說明家暴始末', '高嘉瑜遭到男友林秉樞暴打', '她在遭毆後3天前往醫院驗傷', '只見她除了右眼嚴重瘀青、', '嘴唇、嘴角都有傷口', '右臉腫了半邊', '脖子還有出現勒痕', '雙腳和雙手關節也有傷勢', '的兩腳膝蓋更破皮、流血']


In [39]:
from sentence_transformers import SentenceTransformer, util

SENTENCE_TRANSFORMER_PATH = 'D:/workspace/language_models/distiluse-base-multilingual-cased-v2/'
st = SentenceTransformer(SENTENCE_TRANSFORMER_PATH)

N_TERMS = 10

term_embeddings = st.encode(list(tfreq['term'].iloc[:N_TERMS]))
term_weights = list(tfreq['score'].iloc[:N_TERMS])

def evaluate_sentence_embedding(se, term_embeddings, term_weights):
    import numpy as np
    score = 0
    for i in range(len(term_embeddings)):
        score+=np.dot(se, term_embeddings[i])*term_weights[i]
    score = score/sum(term_weights)
    return(score)

sent_scores = []
for s in sentences:
    se = st.encode(s)
    sent_scores.append(evaluate_sentence_embedding(se, term_embeddings, term_weights))
    
results = pd.DataFrame({'sentence':sentences, 'score':sent_scores})
print(results)

              sentence     score
0               完整新聞標題  0.075799
1          高嘉瑜遭男友林秉樞暴打  0.054871
2                驚悚傷勢照  0.092145
3               完整新聞內文  0.075648
4   民進黨立委高嘉瑜驚傳遭男友林秉樞虐打  0.039276
5      導致她右臉、頭部、四肢多處挫傷  0.007668
6         她昨日向男方提告傷害罪後  0.001679
7            在律師李永裕陪同下  0.062693
8             在立法院議場廣場  0.071100
9                召開記者會  0.097837
10            對外說明家暴始末  0.029892
11        高嘉瑜遭到男友林秉樞暴打  0.052798
12       她在遭毆後3天前往醫院驗傷 -0.010710
13        只見她除了右眼嚴重瘀青、  0.009590
14           嘴唇、嘴角都有傷口  0.027129
15              右臉腫了半邊  0.036700
16            脖子還有出現勒痕  0.037781
17         雙腳和雙手關節也有傷勢 -0.000860
18         的兩腳膝蓋更破皮、流血  0.010876


In [40]:
print(results.sort_values('score', ascending=False))

              sentence     score
9                召開記者會  0.097837
2                驚悚傷勢照  0.092145
0               完整新聞標題  0.075799
3               完整新聞內文  0.075648
8             在立法院議場廣場  0.071100
7            在律師李永裕陪同下  0.062693
1          高嘉瑜遭男友林秉樞暴打  0.054871
11        高嘉瑜遭到男友林秉樞暴打  0.052798
4   民進黨立委高嘉瑜驚傳遭男友林秉樞虐打  0.039276
16            脖子還有出現勒痕  0.037781
15              右臉腫了半邊  0.036700
10            對外說明家暴始末  0.029892
14           嘴唇、嘴角都有傷口  0.027129
18         的兩腳膝蓋更破皮、流血  0.010876
13        只見她除了右眼嚴重瘀青、  0.009590
5      導致她右臉、頭部、四肢多處挫傷  0.007668
6         她昨日向男方提告傷害罪後  0.001679
17         雙腳和雙手關節也有傷勢 -0.000860
12       她在遭毆後3天前往醫院驗傷 -0.010710


In [60]:
stop_words = ['ETtoday','CrazyWinnie', 'ETTODAY', 'YAHOO新聞', '-----', '東森新聞']

sent = []
sent_scores = []
# Loop through articles
for i in range(3):
    article = posts['content'].iloc[i]
    sentences = article_to_sentences(article)
    # Loop through sentences
    for s in sentences:
        if not s in stop_words:
            se = st.encode(s)
            sent.append(s)
            sent_scores.append(evaluate_sentence_embedding(se, term_embeddings, term_weights))
    # Check total number of sentences
    if len(sent_scores)>=10:
        break

results = pd.DataFrame({'sentence':sent, 'score':sent_scores})
print(results.sort_values('score', ascending=False))

              sentence     score
9                召開記者會  0.097837
2                驚悚傷勢照  0.092145
0               完整新聞標題  0.075799
3               完整新聞內文  0.075648
8             在立法院議場廣場  0.071100
7            在律師李永裕陪同下  0.062693
1          高嘉瑜遭男友林秉樞暴打  0.054871
11        高嘉瑜遭到男友林秉樞暴打  0.052798
4   民進黨立委高嘉瑜驚傳遭男友林秉樞虐打  0.039276
16            脖子還有出現勒痕  0.037781
15              右臉腫了半邊  0.036700
10            對外說明家暴始末  0.029892
14           嘴唇、嘴角都有傷口  0.027129
18         的兩腳膝蓋更破皮、流血  0.010876
13        只見她除了右眼嚴重瘀青、  0.009590
5      導致她右臉、頭部、四肢多處挫傷  0.007668
6         她昨日向男方提告傷害罪後  0.001679
17         雙腳和雙手關節也有傷勢 -0.000860
12       她在遭毆後3天前往醫院驗傷 -0.010710
